In [1]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)
cap.set(10,150)

myColors = [[6,95,219,17,195,255],
            [33,81,187,81,250,255],
            [36,144,173,117,255,255],
            [16,77,192,39,211,255]]

myColorValues = [[51,153,255],[0,255,0],[255,0,0],[0,255,255]]        #BGR not RGB

myPoints =  []         # [x,y,colorId]

def findColor(img,myColors,myColorValues):
    imgHSV = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    count = 0
    newPoints = []
    for color in myColors:
        lower = np.array(color[0:3])
        upper = np.array(color[3:6])
        mask = cv2.inRange(imgHSV,lower,upper)
        x,y = getContours(mask)
        cv2.circle(imgres,(x,y),7,myColorValues[count],cv2.FILLED)
        if x!=0 and y!=0:
            newPoints.append([x,y,count])
        count = count + 1
        # cv2.imshow(str(color[3]),mask)\
    return newPoints
        


def getContours(img):
    contours,hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    x,y,w,h = 0,0,0,0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area>500:
            # cv2.drawContours(imgres,cnt,-1,(255,0,0),3)
            peri = cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,0.02*peri,True)
            x,y,w,h = cv2.boundingRect(approx)
    return x+w//2,y

def drawOnCanvas(myPoints,myColorValues):
    for point in myPoints:
        cv2.circle(imgres,(point[0],point[1]),7,myColorValues[point[2]],cv2.FILLED)

while True:
    success,img = cap.read()
    imgres = img.copy()
    cv2.imshow("Video",img)
    newPoints = findColor(img,myColors,myColorValues)
    if(len(newPoints)!=0):
        for newP in newPoints:
            myPoints.append(newP)
    if(len(myPoints)!=0):
        drawOnCanvas(myPoints,myColorValues)
    cv2.imshow("Res",imgres)
    key = cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()